In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import pandas as pd
import numpy as np
import os
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import re
import glob

In [ ]:
%matplotlib inline

In [ ]:
event_path = './csv_for_frame_extraction/csv_for_folder_KP155.800-186.495_D.csv'

In [ ]:
videos_path = '/media/data/astamoulakatos/Survey-2-2012/Project 1/IC2/'

In [ ]:
ending = '*95_D'

In [ ]:
def read_and_prepare_csv(event_path, videos_path, ending):
    df = pd.read_csv(event_path)
    for name in glob.glob(videos_path + ending):
        print(name)
    for n in glob.glob(name + '/**/*Ch2.mpg'):
        print(n)
    codes = ['ANS','ANE']#,'AN']
    df_new = df[df['Observation Code'].isin(codes)]
    df_new.to_csv('test.csv', index=False)
    df_new = df_new.reset_index(drop=True)
    return df_new, name

In [ ]:
def frame_extraction(center_video_path, path_to_save, i, df):
    m = re.search('tos/(.+?).mpg', center_video_path)
    if m:
        name = m.group(1)
    name = name.replace('/','__')    
    path_to_save = path_to_save + name + '__' + str(int(i/2))
    if not os.path.exists(path_to_save):
        os.mkdir(path_to_save)

        path_to_save = path_to_save + '/'

        start = float(df_new['offset'][i])*1000 + 20 
        end = float(df_new['offset'][i+1])*1000 - 100
        cap = cv2.VideoCapture(center_video_path)
        cap.set(cv2.CAP_PROP_POS_MSEC, start)
        #success,image = cap.read()
        s = cap.get(cv2.CAP_PROP_POS_FRAMES)
        cap.set(cv2.CAP_PROP_POS_MSEC, end)
        #success,image = cap.read()
        e = cap.get(cv2.CAP_PROP_POS_FRAMES)
        frames = np.arange(s, e, 4)
        #if not os.path.exists(path_to_save):
        for j in range(len(frames)):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[j])
            suc,im = cap.read()
            cv2.imwrite(path_to_save + 'frame' + str(j) + '.png', im)


In [ ]:
df_new,video = read_and_prepare_csv(event_path, videos_path, ending)

In [ ]:
df_new.head()

In [ ]:
video

In [ ]:
for i in tqdm(range(0,len(df_new)-1,1)): 
    
    if (df['folder'][i] == df['folder'][i+1]):

        center_video = os.path.join(video,df['folder'][i])
        for name in glob.glob(center_video + '/*Ch2.mpg'):
            center_video_path = name

        #print(center_video_path)
        if (df_new['Observation Code'][i] == 'ANS') & (df_new['Observation Code'][i+1] == 'ANE'):
            #if (df_new['KP'][i] < df_new['KP'][i+1]):
            print('Hi!')
            path_to_save = '/media/scratch/astamoulakatos/nsea_video_jpegs/exp_and/'

            frame_extraction(center_video_path, path_to_save, i, df_new)

#         if (df_new['Observation Code'][i] == 'SUE') & (df_new['Observation Code'][i+1] == 'SUS'):
#             if (df_new['KP'][i] > df_new['KP'][i+1]):
#                 print('yo!')
#                 path_to_save = '/media/scratch/astamoulakatos/nsea_video_jpegs/exp_fs/'

#                 frame_extraction(center_video_path, path_to_save, i, df_new)